# 🏥 Prévisions par Service - Pitié-Salpêtrière

Analyse et prévision de l'activité par service hospitalier.

**Services analysés:**
- Médecine (742 lits)
- Chirurgie (385 lits)
- Réanimation (104 lits)
- Soins Intensifs (70 lits)
- USC (49 lits)
- Urgences (~80 box)
- Obstétrique (48 lits)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

DATA_DIR = Path('../data')
print("✅ Bibliothèques chargées")

## 1. Chargement des données

In [ ]:
# Charger historique par service
df = pd.read_csv(DATA_DIR / 'historique_services_2020_2024.csv')
df['date'] = pd.to_datetime(df['date'] + '-01')

# Charger config
with open(DATA_DIR / 'config_services.json') as f:
    config = json.load(f)

print(f"📊 Données chargées: {len(df)} lignes")
print(f"📅 Période: {df['date'].min().strftime('%Y-%m')} → {df['date'].max().strftime('%Y-%m')}")
print(f"\n🏥 Services ({len(config['services'])}): {', '.join(config['services'])}")

## 2. Évolution des lits par service

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

colors = {
    'Médecine': '#3498db',
    'Chirurgie': '#e74c3c',
    'Réanimation': '#9b59b6',
    'Soins Intensifs': '#f39c12',
    'USC': '#1abc9c',
    'Urgences': '#e67e22',
    'Obstétrique': '#95a5a6'
}

# 1. Évolution des lits
ax1 = axes[0, 0]
for service in ['Médecine', 'Chirurgie', 'Réanimation']:
    data = df[df['service'] == service]
    ax1.plot(data['date'], data['lits_installes'], label=service, color=colors[service], linewidth=2)
ax1.axvline(pd.Timestamp('2020-03-01'), color='red', linestyle='--', alpha=0.5, label='COVID')
ax1.axvline(pd.Timestamp('2022-01-01'), color='orange', linestyle='--', alpha=0.5, label='Restructurations')
ax1.set_title('Évolution des lits - Services principaux', fontweight='bold', fontsize=12)
ax1.set_ylabel('Nombre de lits')
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3)

# 2. Soins critiques
ax2 = axes[0, 1]
for service in ['Réanimation', 'Soins Intensifs', 'USC']:
    data = df[df['service'] == service]
    ax2.plot(data['date'], data['lits_installes'], label=service, color=colors[service], linewidth=2)
ax2.axvline(pd.Timestamp('2020-03-01'), color='red', linestyle='--', alpha=0.5)
ax2.set_title('Évolution des lits - Soins critiques', fontweight='bold', fontsize=12)
ax2.set_ylabel('Nombre de lits')
ax2.legend()
ax2.grid(True, alpha=0.3)

# 3. Taux d'occupation
ax3 = axes[1, 0]
for service in ['Médecine', 'Chirurgie', 'Urgences']:
    data = df[df['service'] == service]
    ax3.plot(data['date'], data['taux_occupation'], label=service, color=colors[service], linewidth=2)
ax3.axhline(85, color='orange', linestyle='--', alpha=0.7, label='Seuil tension (85%)')
ax3.axhline(95, color='red', linestyle='--', alpha=0.7, label='Seuil critique (95%)')
ax3.set_title('Taux d\'occupation', fontweight='bold', fontsize=12)
ax3.set_ylabel('Taux (%)')
ax3.set_ylim(30, 120)
ax3.legend(loc='lower right')
ax3.grid(True, alpha=0.3)

# 4. Admissions mensuelles
ax4 = axes[1, 1]
df_pivot = df.pivot_table(index='date', columns='service', values='admissions_mensuelles')
df_pivot[['Médecine', 'Chirurgie']].plot(ax=ax4, color=[colors['Médecine'], colors['Chirurgie']], linewidth=2)
ax4.set_title('Admissions mensuelles - Médecine & Chirurgie', fontweight='bold', fontsize=12)
ax4.set_ylabel('Admissions')
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../data/graphs/evolution_services.png', dpi=150)
plt.show()

## 3. Modèle de prévision par service

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

def prepare_features(df_service):
    """Prépare les features pour un service."""
    df = df_service.copy().sort_values('date')
    
    # Features temporelles
    df['month'] = df['date'].dt.month
    df['year'] = df['date'].dt.year
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    df['trend'] = range(len(df))
    
    # Lag features
    df['lag_1'] = df['admissions_mensuelles'].shift(1)
    df['lag_12'] = df['admissions_mensuelles'].shift(12)  # même mois année précédente
    df['rolling_3'] = df['admissions_mensuelles'].shift(1).rolling(3).mean()
    
    df = df.dropna()
    return df

def train_predict_service(df, service_name, horizon=6):
    """Entraîne un modèle et prédit pour un service."""
    df_service = df[df['service'] == service_name].copy()
    df_prep = prepare_features(df_service)
    
    feature_cols = ['month', 'year', 'month_sin', 'month_cos', 'trend', 'lag_1', 'lag_12', 'rolling_3']
    
    # Split
    split = int(len(df_prep) * 0.8)
    train = df_prep.iloc[:split]
    test = df_prep.iloc[split:]
    
    X_train = train[feature_cols]
    y_train = train['admissions_mensuelles']
    X_test = test[feature_cols]
    y_test = test['admissions_mensuelles']
    
    # Modèle
    model = RandomForestRegressor(n_estimators=50, max_depth=5, random_state=42)
    model.fit(X_train, y_train)
    
    # Prédictions test
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    
    return {
        'service': service_name,
        'model': model,
        'mae': mae,
        'test_dates': test['date'].values,
        'y_test': y_test.values,
        'y_pred': y_pred,
        'last_values': df_prep.iloc[-1][feature_cols].values
    }

# Entraîner pour chaque service
results = {}
print("📊 ENTRAÎNEMENT DES MODÈLES PAR SERVICE:\n")

for service in config['services']:
    res = train_predict_service(df, service)
    results[service] = res
    print(f"   {service:<20} MAE: {res['mae']:>8.1f} admissions/mois")

print("\n✅ Modèles entraînés pour tous les services")

## 4. Prévisions 6 mois

In [ ]:
# Générer prévisions pour les 6 prochains mois
from datetime import datetime
from dateutil.relativedelta import relativedelta

last_date = df['date'].max()
future_months = [last_date + relativedelta(months=i+1) for i in range(6)]

previsions = []

for service in config['services']:
    df_service = df[df['service'] == service].sort_values('date')
    last_admission = df_service['admissions_mensuelles'].iloc[-1]
    last_12 = df_service['admissions_mensuelles'].iloc[-12] if len(df_service) >= 12 else last_admission
    rolling = df_service['admissions_mensuelles'].tail(3).mean()
    
    model = results[service]['model']
    
    for i, date in enumerate(future_months):
        features = [
            date.month,  # month
            date.year,   # year
            np.sin(2 * np.pi * date.month / 12),  # month_sin
            np.cos(2 * np.pi * date.month / 12),  # month_cos
            len(df_service) + i,  # trend
            last_admission,  # lag_1 (simplifié)
            last_12,  # lag_12
            rolling   # rolling_3
        ]
        
        pred = model.predict([features])[0]
        
        # Mise à jour pour prochaine itération
        rolling = (rolling * 2 + pred) / 3
        last_admission = pred
        
        previsions.append({
            'date': date.strftime('%Y-%m'),
            'service': service,
            'admissions_prevues': int(pred),
            'intervalle_bas': int(pred * 0.9),
            'intervalle_haut': int(pred * 1.1)
        })

df_prev = pd.DataFrame(previsions)
df_prev.to_csv('../data/previsions_services_6mois.csv', index=False)

print("📊 PRÉVISIONS 6 MOIS PAR SERVICE:\n")
pivot = df_prev.pivot_table(index='date', columns='service', values='admissions_prevues')
print(pivot.to_string())

print(f"\n✅ Fichier sauvegardé: previsions_services_6mois.csv")

## 5. Visualisation des prévisions

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

services_plot = ['Médecine', 'Chirurgie', 'Réanimation', 'Urgences']

for idx, service in enumerate(services_plot):
    ax = axes[idx // 2, idx % 2]
    
    # Historique
    hist = df[df['service'] == service].tail(24)  # 2 dernières années
    ax.plot(hist['date'], hist['admissions_mensuelles'], 'b-', linewidth=2, label='Historique')
    
    # Prévisions
    prev = df_prev[df_prev['service'] == service]
    prev_dates = pd.to_datetime(prev['date'] + '-01')
    ax.plot(prev_dates, prev['admissions_prevues'], 'r--', linewidth=2, label='Prévision')
    ax.fill_between(prev_dates, prev['intervalle_bas'], prev['intervalle_haut'], 
                    color='red', alpha=0.2, label='Intervalle 90%')
    
    ax.set_title(f'{service}', fontweight='bold', fontsize=12)
    ax.set_ylabel('Admissions/mois')
    ax.legend(loc='upper left')
    ax.grid(True, alpha=0.3)
    
    # Stats
    lits = config['services_details'][service]['lits_2023']
    ax.text(0.98, 0.02, f'Capacité: {lits} lits', transform=ax.transAxes, 
            ha='right', fontsize=9, style='italic')

plt.suptitle('Prévisions d\'activité par service - 6 prochains mois', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig('../data/graphs/previsions_services.png', dpi=150)
plt.show()

## 6. Alertes et recommandations

In [ ]:
print("="*70)
print("🚨 ALERTES ET RECOMMANDATIONS")
print("="*70)

for service in config['services']:
    prev = df_prev[df_prev['service'] == service]
    details = config['services_details'][service]
    lits = details['lits_2023']
    taux_ref = details['taux_occupation_ref']
    
    # Calcul charge prévisionnelle
    adm_moy = prev['admissions_prevues'].mean()
    dms = 5  # Durée moyenne séjour estimée
    lits_necessaires = (adm_moy / 30) * dms
    taux_prevu = (lits_necessaires / lits * 100) if lits > 0 else 0
    
    print(f"\n🏥 {service}:")
    print(f"   Capacité: {lits} lits")
    print(f"   Admissions prévues: {int(adm_moy)}/mois")
    
    if taux_prevu > 95:
        print(f"   ⚠️  ALERTE CRITIQUE: Surcharge prévisible ({taux_prevu:.0f}%)")
        print(f"   💡 Recommandation: Ouvrir {int(lits_necessaires - lits * 0.85)} lits supplémentaires")
    elif taux_prevu > 85:
        print(f"   ⚡ TENSION: Occupation élevée prévisible ({taux_prevu:.0f}%)")
        print(f"   💡 Recommandation: Prévoir renforts personnel")
    elif taux_prevu < 60:
        print(f"   📉 SOUS-UTILISATION: Occupation faible ({taux_prevu:.0f}%)")
        print(f"   💡 Recommandation: Optimiser ou réaffecter ressources")
    else:
        print(f"   ✅ Situation normale ({taux_prevu:.0f}% prévu)")

## 7. Export pour dashboard

In [ ]:
# Créer JSON pour l'API/Dashboard
dashboard_data = {
    "derniere_maj": datetime.now().strftime('%Y-%m-%d %H:%M'),
    "services": {}
}

for service in config['services']:
    hist = df[df['service'] == service].tail(12)
    prev = df_prev[df_prev['service'] == service]
    details = config['services_details'][service]
    
    dashboard_data['services'][service] = {
        "capacite": {
            "lits": details['lits_2023'],
            "places_ambu": details['places_ambu']
        },
        "historique_12_mois": {
            "dates": hist['date'].dt.strftime('%Y-%m').tolist(),
            "admissions": hist['admissions_mensuelles'].tolist(),
            "taux_occupation": hist['taux_occupation'].tolist()
        },
        "previsions_6_mois": {
            "dates": prev['date'].tolist(),
            "admissions": prev['admissions_prevues'].tolist(),
            "intervalle_bas": prev['intervalle_bas'].tolist(),
            "intervalle_haut": prev['intervalle_haut'].tolist()
        },
        "mae_modele": round(results[service]['mae'], 1)
    }

with open('../data/dashboard_services.json', 'w') as f:
    json.dump(dashboard_data, f, indent=2, ensure_ascii=False)

print("✅ Export dashboard: data/dashboard_services.json")
print(f"\n📊 Données disponibles pour {len(config['services'])} services")
print("   → Historique 12 mois")
print("   → Prévisions 6 mois avec intervalles de confiance")
print("   → Métriques de fiabilité (MAE)")